# Click Through Rate Prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
# import seaborn as sns
# sns.set()
import random
from catboost import CatBoostClassifier

In [22]:
file_path = "../CTR/train/train.csv"
df = pd.read_csv(file_path, header=0, nrows=10000000).sample(frac=1)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
9226611,5.825158e+16,0,14102221,1005,1,cbb01868,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,17037,320,50,1934,2,39,-1,16
8676788,1.539006e+19,0,14102218,1005,1,222acd54,de0f0f82,f028772b,ecad2386,7801e8d9,...,1,0,21878,320,50,2526,0,35,100081,221
4249905,1.809670e+19,1,14102201,1002,0,c6af341d,19944fab,50e219e0,ecad2386,7801e8d9,...,0,0,21703,320,50,2497,3,43,100151,42
6457053,1.592868e+19,0,14102210,1005,0,dcc019de,250469f8,3e814130,ecad2386,7801e8d9,...,1,0,21768,320,50,2506,0,35,100075,157
8656391,1.374903e+19,0,14102218,1005,1,5ee41ff2,17d996e6,f028772b,ecad2386,7801e8d9,...,1,0,16920,320,50,1899,0,431,-1,117


## Checking for null values

In [ ]:
df.isnull().values.any()

False

## Calculating the probability of ad getting clicked

In [ ]:
df["click"].value_counts() / len(df)

0    0.833864
1    0.166136
Name: click, dtype: float64

In [ ]:
df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [ ]:
len(df.columns)

24

In [23]:
train_count = int(len(df) * 0.8)
x_train = df.iloc[:train_count,2:]
y_train = df.iloc[:train_count,1]
x_test = df.iloc[train_count: ,2:]
y_test = df.iloc[train_count: ,1]

In [24]:
catBoost_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [25]:
catBoost_model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.5,
    task_type='GPU',
    loss_function='Logloss',
)

In [26]:
catBoost_model.fit(
    x_train, y_train,
    eval_set=(x_test, y_test),
    cat_features=catBoost_features,
    verbose=10,
)

0:	learn: 0.4790276	test: 0.4648886	best: 0.4648886 (0)	total: 334ms	remaining: 16.4s
10:	learn: 0.3883492	test: 0.3821309	best: 0.3821309 (10)	total: 3.46s	remaining: 12.3s
20:	learn: 0.3839891	test: 0.3772471	best: 0.3772471 (20)	total: 6.49s	remaining: 8.97s
30:	learn: 0.3814134	test: 0.3746781	best: 0.3746781 (30)	total: 9.61s	remaining: 5.89s
40:	learn: 0.3797363	test: 0.3729851	best: 0.3729851 (40)	total: 12.8s	remaining: 2.81s
49:	learn: 0.3790332	test: 0.3723265	best: 0.3723265 (49)	total: 15.6s	remaining: 0us
bestTest = 0.3723264687
bestIteration = 49


In [27]:
pred = catBoost_model.predict(x_test)

In [28]:
print(pred[10])

0


In [29]:
catBoost_model.score(x_test,y_test)

0.8421655